In [10]:
import os
from skimage.metrics import structural_similarity as ssim
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
# from scipy.optimize import linprog

In [11]:
def eve_distortion(eve_intensities): #from the reconstructed image in uint8 type
    feve_intensities = eve_intensities.astype(np.float)
    fintensities = intensities.astype(np.float)
    dif = (fintensities -  feve_intensities)
    eve_dist_mat = dif**2
    occurance_sum = height*width #?!
    eve_dist = eve_dist_mat.sum()/occurance_sum
    return eve_dist

# Prepare Image Data

In [12]:
image_name = 'person'
img_file = image_name + '.tif'
intensities = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
height = np.shape(intensities)[0]
width = np.shape(intensities)[1]
imf = np.float32(intensities)

'''From scratch'''
#     all_nq_ac = []
#     all_dcs = []
#     for i in range(0,int(height/8)):
#         print(i)
#         for j in range(0,int(width/8)):
#             blk = imf[8*i:8*(i+1), 8*j:8*(j+1)]
#             shifted_blk = blk - 128 #level_shift for dct
#             dct = cv2.dct(shifted_blk)
#             nq_coef = np.round(dct)
#             nq_coef_dc = nq_coef[0,0]
#             nq_coef_ac = np.delete(nq_coef, [0,0])

#             all_dcs = np.append(all_dcs, nq_coef_dc)
#             all_nq_ac = np.append(all_nq_ac, nq_coef_ac)


#     dcs_file_name = image_name + '_dcs.npy'        
#     np.save(dcs_file_name, all_dcs)        
#     acs_file_name = image_name + '_acs.npy'        
#     np.save(acs_file_name, all_nq_ac)
'''...........................................'''

''' If saved already, so comment the dct part'''
dcs_file_name = image_name + '_dcs.npy' 
acs_file_name = image_name + '_acs.npy'
all_dcs = np.load(dcs_file_name)
all_nq_ac = np.load(acs_file_name)
'''...........................................'''

'...........................................'

In [13]:
ac_values, ac_counts = np.unique(all_nq_ac, return_counts=True)
ac_values_dic = {}
ac_counts_dic = {}
ac_bits_all_sizes = []
for i in range(1,11):
    neg_part = range(-(2**i-1), -2**(i-1)+1)
    pos_part = range(2**(i-1), 2**i)
    values_in_size = np.append(neg_part,pos_part)
    ac_values_dic[i] = values_in_size
    
    counts_in_size = np.zeros(np.size(values_in_size))
    for j in range(np.size(values_in_size)):
        if values_in_size[j] in ac_values:
            position_in_counts = np.where(
                ac_values == values_in_size[j])
            counts_in_size[j] = ac_counts[position_in_counts]
            
    ac_counts_dic[i] = counts_in_size
    
    #Rate
    ac_bits_in_size = sum(counts_in_size)*i
    ac_bits_all_sizes = np.append(ac_bits_all_sizes, ac_bits_in_size)
total_ac_bits = sum(ac_bits_all_sizes)

ac_prbs = ac_counts/sum(ac_counts)


n0 = 0
pos0 = np.where(ac_values == 0)
ac_values_n0 = ac_values.copy()
ac_counts_n0 = ac_counts.copy()
ac_values_n0 = np.delete(ac_values_n0, pos0)
ac_counts_n0 = np.delete(ac_counts_n0, pos0)


# select 'num' from the middle and sides
num = 256
part1 = ac_values_n0[0:int(num/4)]
neg1_ind = np.where(ac_values_n0 == -1)[0]
part2 = ac_values_n0[int(neg1_ind-int(num/4)+1):\
                     int(neg1_ind+int(num/4)+1)]
part3 = np.append(ac_values_n0[-int(num/4):-1], ac_values_n0[-1])
selected_values_1 = np.append(part1, part2)
selected_values = np.append(selected_values_1, part3)

part1_c = ac_counts_n0[0:int(num/4)]
part2_1_c = ac_counts_n0[int(neg1_ind-int(num/4)+1):int(neg1_ind+1)]
part2_2_c = ac_counts_n0[int(neg1_ind+1):int(neg1_ind+int(num/4)+1)]
part2_c = np.append(part2_1_c, part2_2_c)
part3_c = np.append(ac_counts_n0[-int(num/4):-1], ac_counts_n0[-1])
selected_counts_1 = np.append(part1_c, part2_c)
selected_counts = np.append(selected_counts_1, part3_c)

org_ac_values_n0 = ac_values_n0.copy()
org_ac_counts_n0 = ac_counts_n0.copy()

# Taking care of not selected values, their pairs
ns_part1 = ac_values_n0[int(num/4):int(neg1_ind-int(num/4)+1)]
ns_part2 = ac_values_n0[int(neg1_ind+int(num/4)+1):\
                       -int(num/4)]
ns_values = np.append(ns_part1, ns_part2)

ns_pairs_elements1 = []
ns_pairs_elements2 = []
no_pair_elements_neg = []
no_pair_elements_pos = []
for i in ns_part1:
    i_size = np.floor(np.log2(abs(i)))+1
    bs_pair = i+2**i_size-1+2**(i_size-1)
    if bs_pair in ns_part2:
        ns_pairs_elements1 = np.append(ns_pairs_elements1,i)
        ns_pairs_elements2 = np.append(ns_pairs_elements2,bs_pair)
    else:
        no_pair_elements_neg = np.append(no_pair_elements_neg, i)

for i in ns_part2:
    if not i in ns_pairs_elements2:
        no_pair_elements_pos = np.append(no_pair_elements_pos, i)
        
no_pair_min_len = min(len(no_pair_elements_neg),\
                      len(no_pair_elements_pos))
no_pair_elements1 = no_pair_elements_neg[0:no_pair_min_len]
no_pair_elements2 = no_pair_elements_pos[0:no_pair_min_len]

extra_elements_1 =  no_pair_elements_neg[no_pair_min_len:\
                                        len(no_pair_elements_neg)]
extra_elements_2 =  no_pair_elements_pos[no_pair_min_len:\
                                        len(no_pair_elements_pos)]
extra_elements = np.append(extra_elements_1, extra_elements_2)

extra_half_ind = int(np.floor(len(extra_elements)/2))
extra_elements1 = extra_elements[0:extra_half_ind]
# extra_elements2 = extra_elements[extra_half_ind:]
extra_elements2 = extra_elements[extra_half_ind:-1]

sac_values_n0 = selected_values
sac_counts_n0 = selected_counts

var = sac_values_n0.copy()
prbs = sac_counts_n0/sum(sac_counts_n0)

# Optimization Parameters

# Creating Model

# List of Lambdas

In [14]:
Lambdas1 = np.arange(0, 100000, 10000)
Lambdas2 = np.arange(100000, 160000, 5000)
Lambdas3 = np.arange(160000, 192730, 1000)
Lambdas4 = [192730,192731]
Lambdas5 = np.arange(193000, 200000, 500)
Lambdas6 = np.arange(200000, 300000, 5000)
Lambdas7 = np.arange(300000, 700000, 50000)

Lambdas = np.concatenate(
    (Lambdas1, Lambdas2, Lambdas3, Lambdas4,
     Lambdas5, Lambdas6, Lambdas7), axis = 0)
problematic_lambdas = np.array([70000, 80000, 167000, 185000])
Lambdas = np.setdiff1d(Lambdas, problematic_lambdas)
len(Lambdas)

95

# Reconstructions, PSNRs

In [15]:
Lambdas = [100000]
for Lambda in Lambdas:
# Better MSB 0 assignment ##################################### 
    pairs_show_file_name = image_name + '_pairs_' +str(Lambda)+'.npy' 
    mip_pairs_show = np.load(pairs_show_file_name)

    pairs_show = np.transpose(mip_pairs_show)
    abs_pairs_show = abs(pairs_show)
    pair_diff = abs(abs_pairs_show [:,0] - abs_pairs_show [:,1])
    pair_diff = pair_diff[..., None]
    pairs_diff = np.append(pairs_show, pair_diff, axis = 1)
    pairs_diff_sort = pairs_diff[pairs_diff[:,2].argsort()[::-1]]

    msb0_elements = np.zeros(np.shape(mip_pairs_show)[1])
    msb1_elements = np.zeros(np.shape(mip_pairs_show)[1])

    opt_pairs_prbs = []
    opt_pairs_prbs_dic = {}

    ac_dists = []
    for i in range(0,np.shape(mip_pairs_show)[1]):
        pair1 = pairs_diff_sort[i][0]
        pos1 = np.where(ac_values_n0 == pair1)
        prb_pair1 = prbs[pos1]

        pair2 = pairs_diff_sort[i][1]
        pos2 = np.where(ac_values_n0 == pair2)
        prb_pair2 = prbs[pos2]

        if not np.mod(i,2):
            if abs(pair1) < abs(pair2):
                msb0_elements [i] = pair1
                msb1_elements [i] = pair2
            else:
                msb0_elements [i] = pair2
                msb1_elements [i] = pair1
        if np.mod(i,2):
            if abs(pair1) > abs(pair2):
                msb0_elements [i] = pair1
                msb1_elements [i] = pair2
            else:
                msb0_elements [i] = pair2
                msb1_elements [i] = pair1   


        # probability of pairs
        pair_prb = prb_pair1 + prb_pair2
        opt_pairs_prbs = np.append(opt_pairs_prbs, pair_prb)
        element1 = pair1
        opt_pairs_prbs_dic[element1] = pair_prb

        # distortion between ac pairs
        ac_dist = 0.5*(pair1-pair2)**2*pair_prb
        ac_dists = np.append(ac_dists, ac_dist)

    ac_dist_total = sum(ac_dists)
    ac_dist_total_db = 10*np.log10(ac_dist_total)
    ac_dist_total_dbs = np.append(ac_dist_total_dbs, ac_dist_total_db)


    #     ###### For Lambdas
    opt_pairs_prbs_lambda_dic [Lambda] = opt_pairs_prbs_dic


    # Reconstruction
    eve_msb0_intensities = np.zeros([height,width])
    eve_msb1_intensities = np.zeros([height,width])
    for i in range(0,int(height/8)):
        for j in range(0,int(width/8)):
            blk = imf[8*i:8*(i+1), 8*j:8*(j+1)]
            shifted_blk = blk - 128 #level_shift for dct
            dct = cv2.dct(shifted_blk)
            nq_coef = np.round(dct)
            nq_coef_ac = np.delete(nq_coef, [0,0])
            nq_coef_dc = nq_coef[0,0]

            eve_ac_msb0 = nq_coef_ac.copy()
            eve_ac_msb1 = nq_coef_ac.copy()
            for m in range(np.size(nq_coef_ac)):
                if nq_coef_ac[m] in msb0_elements:
                    eve_ac_msb0[m] =  nq_coef_ac[m]

                    pos = np.where(msb0_elements == nq_coef_ac[m])
    #                     poss = np.where(msb0_elements == nq_coef_ac[m])
    #                     pos = poss[0][0]

                    eve_ac_msb1[m] = msb1_elements[pos]
                if nq_coef_ac[m] in msb1_elements:
                    eve_ac_msb1[m] =  nq_coef_ac[m]

                    pos = np.where(msb1_elements == nq_coef_ac[m])
    #                     poss = np.where(msb1_elements == nq_coef_ac[m])
    #                     pos = poss[0][0]
                    eve_ac_msb0[m] = msb0_elements[pos]


            eve_msb0_coef1 = np.insert(eve_ac_msb0, 0, 128)
#             eve_msb0_coef1 = np.insert(eve_ac_msb0, 0, nq_coef_dc)
            eve_msb0_coef = np.reshape(eve_msb0_coef1,[8,8])

            eve_msb0_blk1 = cv2.idct(eve_msb0_coef)
            eve_msb0_blk2 = eve_msb0_blk1+128 # level_shift after idct
            eve_msb0_blk = np.round(eve_msb0_blk2)  
            eve_msb0_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_msb0_blk


            eve_msb0_coef1 = np.insert(eve_ac_msb0, 0, 128)
#             eve_msb1_coef1 = np.insert(eve_ac_msb1, 0, nq_coef_dc)
            eve_msb1_coef = np.reshape(eve_msb1_coef1,[8,8])

            eve_msb1_blk1 = cv2.idct(eve_msb1_coef)
            eve_msb1_blk2 = eve_msb1_blk1+128 # level_shift after idct
            eve_msb1_blk = np.round(eve_msb1_blk2)  
            eve_msb1_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_msb1_blk


    eve_msb0_modified = eve_msb0_intensities.copy()
    eve_msb0_modified[eve_msb0_modified<0] = 0
    eve_msb0_modified[eve_msb0_modified>255] = 255
    #eve_msb0_uint8 = eve_msb0_intensities.astype(np.uint8)
    filename = image_name +'_msb0_'+str(Lambda)+'.png'
    cv2.imwrite(filename, eve_msb0_modified)
    
    eve_msb1_modified = eve_msb1_intensities.copy()
    eve_msb1_modified[eve_msb1_modified<0] = 0
    eve_msb1_modified[eve_msb1_modified>255] = 255
#     eve_msb1_uint8 =eve_msb1_intensities.astype(np.uint8)
    filename = image_name +'_msb1_'+str(Lambda)+'.png'
    cv2.imwrite(filename, eve_msb1_modified)

    # Distortion
    eve_msb0_dist_db = 10*np.log10(eve_distortion(eve_msb0_uint8))
    eve_msb1_dist_db = 10*np.log10(eve_distortion(eve_msb1_uint8))

    psnr_msb0_db = 20*np.log10(255)-eve_msb0_dist_db
    psnr_msb1_db = 20*np.log10(255)-eve_msb1_dist_db

    avg_psnr = \
    20*np.log10(255)-10*np.log10((eve_distortion(eve_msb0_uint8)
                                 +eve_distortion(eve_msb1_uint8))/2)
    avg_psnrs = np.append(avg_psnrs, avg_psnr)
    psnr_msb0s = np.append(psnr_msb0s, psnr_msb0_db)
    psnr_msb1s = np.append(psnr_msb1s, psnr_msb1_db)

IndexError: index 259 is out of bounds for axis 0 with size 256